In [4]:
! pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 23.8 MB/s eta 0:00:00


In [5]:
from google.colab import drive
import mne
from mne.preprocessing import ICA
import os

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/SEED/EEG_emotion_classification/Final_epochs_pre_ICA/'
filename = 'P1E1-epo.fif'

In [8]:
epochs = mne.read_epochs(os.path.join(path, filename), preload=True)

ica = ICA(random_state=97)
ica.fit(epochs)
ica.plot_components(picks='eeg', )

ica.apply(epochs)

Output hidden; open in https://colab.research.google.com to view.